This exercise will require to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, It is required to register a free account on the http://www.quandl.com website.

After registration,a unique API key will be provided, that should be stored:

In [3]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = #API-key goes here


Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer to make use of the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package,we are not going to use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since we won't have access to DataFrames,it is encouraged to use Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

It is to be Kept in mind that the JSON responses will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so it makes sense to read up on indexing dictionaries in the documentation provided above.

In [6]:
# First, import the relevant modules
import requests
import json
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'limit_output': 100})

In [27]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-02collapse=daily&order=asc&api_key=' + API_KEY)

In [28]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2017-01-01',
  'end_date': '2017-01-02',
  'frequency': 'daily',
  'data': [['2017-01-02',
    34.99,
    35.94,
    34.99,
    35.8,
    None,
    44700.0,
    1590561.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': 'asc'}}

These are the tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Solution 1

In [9]:
res = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&collapse=daily&order=asc&api_key=' + API_KEY)


Making use of the requests library of python to interact with Quandl API.The url parameters consists of the code for the dataset, a sample of it which would be used to perform the various exercises. The data is to be received in JSON format, for the financial year of 2017.The data is sampled daily for that year and sorted in ascending order.

## Solution 2

In [10]:
res_dict = res.json()


The response received above, is decoded to reveal the underlying structure of the request.It seems to be dict of dict structure.

## Solution 3

In [11]:
dataset_data = res_dict['dataset_data']

The structure has to be accessed layer by layer.Accessing the values to the outermost key 'dataset_data'.

In [12]:
data = dataset_data['data']

Now the relevant data is extracted from inside the structure.It appears to be a 2-D array,with each row having info for the stock trading that happened in any given day

In [13]:
# initialising a dictionary
dataset_dict = {}

In [14]:
for ls in data:
    s=ls[0]
    dataset_dict[s]=ls[1:]
   

Extracting and storing the trading data in a dictionary with each date as the key and all the relevant trading info for that day as its value.

In [15]:
list_columns = dataset_data['column_names']
list_columns

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

Having a peek at what the different columns of the data represent.

In [16]:
opening_prices = []
for values in dataset_dict.values():
    opening_prices.append(values[0])

storing all the stock prices at the value at which they open in a given day in a list.

In [17]:
max(opening_prices)

TypeError: '>' not supported between instances of 'NoneType' and 'float'

On trying to find the highest opening price, it is found that there are some missing values.Hence, an error occured while trying to find the max value. 

Removing all the none values from the list.

In [18]:
opening_prices = [x for x in opening_prices if x!=None]

In [19]:
print(max(opening_prices))
print(min(opening_prices))

53.11
34.0


The highest opening price is \\$53.11 and the lowest is \\$34.00

## Solution 4

In [20]:
change_instock = []
for values in dataset_dict.values():
    change = values[1]-values[2]
    change_instock.append(change)

In [21]:
max(change_instock)

2.8100000000000023

Extracting the data for each date and the change, based on the high's and low's for each day, is stored in a list.The max change is observed to be \\$2.81

## Solution 5

In [22]:
prevday_close = 0
change = []
for values in dataset_dict.values():
    c = values[3]-prevday_close
    change.append(c)
    prevday_close = values[3]



In [23]:
del change[0]


In [24]:
max(change)

1.7199999999999989

Max change between any two days,based on closing price, is noticed to be \\$1.71

## Solution 6

In [25]:
len = 0
sum_tradevol = 0
for values in dataset_dict.values():
    sum_tradevol = sum_tradevol + values[5]
    len=len+1

In [26]:
average_tradevol = sum_tradevol/len
average_tradevol

89124.33725490196

The average daily traded volume is 89124.337 units

Therefore,Solutions to all the tasks have been done.